This notebook can be used to play around with all options and various parameters settings of background subtraction algorithms when using Birdwatcher for movement detection, and see how this influences the results. You can use this notebook to optimize the settings for your own videos.

**NOTE:** this notebook is specifically usefull for finding optimal settings for **location detection**.

In [ ]:
import birdwatcher as bw
import birdwatcher.movementdetection as md
from birdwatcher.plotting import imshow_frame # birdwatcher has vizualization tools

import matplotlib.pyplot as plt
%matplotlib inline

### Select video fragment

Choose a short representative video fragment where the object of interest is moving quite a lot. 

See notebook 1 for more information of how to select a video fragment or a region of interest (roi and nroi).

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zf20s_low.mp4')

# select video fragment
startat = '00:00:00'   # in HOURS:MM:SS
duration = None         # in seconds

# get nframes based on duration
nframes = int(vfs.avgframerate*duration) if duration is not None else None

# specify h1, h2, w1, w2, or choose None to use the whole frame
roi = None   # region of interest
nroi = None   # nót region of interest

Check roi and nroi in frame:

In [ ]:
frame = vfs.iter_frames(startat=startat, nframes=nframes).peek_frame()

if roi is not None:
    frame = vfs.iter_frames(startat=startat, nframes=1).draw_text(['roi'], org=(roi[0],roi[3]))
    imshow_frame(frame.peek_frame(), draw_rectangle=roi)

if nroi is not None:
    frame = vfs.iter_frames(startat=startat, nframes=1).draw_text(['nroi'], org=(nroi[0],nroi[3]))
    imshow_frame(frame.peek_frame(), draw_rectangle=nroi)

Look at the chosen video fragment:

In [ ]:
vfs.iter_frames(startat=startat, nframes=nframes).show(framerate=vfs.avgframerate)

### Choose parameters

In this example, we will use background subtractor MOG2. For more information of the background subtraction algorithms make sure to look at notebook 2. To choose parameters from another algorithm just modify the dictionary below with the appropriate parameters.

To get a better feeling of the effect of the various background subtraction parameters, you can play around with different values. Also, some processing steps before or after performing background subtraction might improve location detection, and therefore, you can compare the settings of those as well.

In the dictionary below, decide which settings you would like by adding one or more values in the list after each parameter.

In [ ]:
settings = {'bgs_params':  {'History': [3, 6, 12],
                            'ComplexityReductionThreshold': [0.05],
                            'BackgroundRatio': [0.1],
                            'NMixtures': [7],
                            'VarInit': [15],
                            'VarMin': [4, 10],
                            'VarMax': [75],
                            'VarThreshold': [30, 70],
                            'VarThresholdGen': [9],
                            'DetectShadows': [False],
                            'ShadowThreshold': [0.5],
                            'ShadowValue': [0]},

            'processing':  {'color': [True, False],   # booleans only
                            'resizebyfactor': [1, (2/3)],   # use '1' for no change in size
                            'blur': [0, 10],   # use '0' for no blur
                            'morphologyex': [True, False]}}   # booleans only

all_combinations = list(md.product_dict(**settings['bgs_params'], **settings['processing']))
print(f'There are {len(all_combinations)} different combinations of settings to perform movement detection.')

The higher the number of combinations, the longer the next step will take. Another option is to start by tweaking some parameters, and fine-tune in next rounds by running this notebook again with different settings.

### Run movemement detection per combination of settings

Movement detection is done for each combination of settings, and the mean coordinate per frame is saved in a Pandas dataframe.

**WARNING:** This step might take a while, depending on the number of settings combinations! To shorten runtime, reduce the number of combinations and/or choose a shorter videofragment.

In [ ]:
%%time
params = md.apply_all_parameters(vfs, settings, bgs_type=bw.BackgroundSubtractorMOG2, 
                                 startat=startat, duration=duration, roi=roi, nroi=nroi,
                                 reportprogress=25)

params.save_parameters(f'output/')

The results are saved in a folder with the name of the VideoFileStream. Also, a readme.txt file with the parameter settings is saved to quickly look up which settings were used.

In [ ]:
params.path

To get the optimal parameter settings, you'll probably do several rounds with a different combination of settings. Then, a new project folder is created with a number added as suffix to the foldername to display the round.

The output of applying all parameters is a `ParameterSelection` object, which contains information of the videofragment and the results of all setting combinations.

In [ ]:
params.get_info()

In [ ]:
params.df

Here, you see a pandas dataframe with in the columns all parameters that are used to run movement detection. The rows show the specific value of each parameter and the resulted mean x,y coordinates per frame (NaN means there were no nonzero pixels found for that frame). 

### Load ParameterSelection

You can run and save `apply_all_parameters`, and later look at the results by loading a `ParameterSelection` object like this:

In [ ]:
params = md.load_parameterselection(f'output\params_zf20s_low')

Make sure the location of the original video where the `ParameterSelection` object is based on, has not changed. Then, it is also possible to load the associated videofilestream directly:

In [ ]:
params.vfs

Or watch the videofragment of which the `ParameterSelection` object is based on:

In [ ]:
frames_fragment = params.get_videofragment()
frames_fragment.show()

To access the data as Pandas dataframe, run:

In [ ]:
params.df

### Correction resizebyfactor

If you've used setting 'resizebyfactor' this has changed the width and height of the frames. Below, we correct for this change in pixel resolution, so that it's easier to see and compare the effects of different settings on the movementdetection results below.

In [ ]:
params.df['pixel'] = params.df['pixel'] / params.df['resizebyfactor']
params.df.loc[:, ('resizebyfactor', 'coords', 'pixel')]

### Visualize results

Before visualizing the results, look again at all settings.

In [ ]:
# the following settings have been used for backgroundsubstraction in this dataframe
params.get_parameters('all')

Here, you see for which settings multiple values have been used to run movement detection. So, these are also the settings that are interesting to compare in plots or superimpose on the video fragment.

In [ ]:
# the following settings have been tested with multiple values
params.get_parameters('multi_only')

#### Plots

First, choose for each parameter with multiple values which value is the default. TIP: you can copy the output dictionary above and choose one of the values in each list. Use the value of which you think will provide the best location detection. If you have no idea, don't worry, just choose one.

In [ ]:
default_values = {'resizebyfactor': 1.0,
                  'color': True,
                  'VarThreshold': 30,
                  'blur': 0,
                  'morphologyex': False,
                  'History': 4,
                  'VarMin': 4}

You can plot the results of two parameters in one figure. The different values of one parameter is outlined in the rows and the other parameter in the columns of the subplots.

In [ ]:
rows = 'blur'
cols = 'color'

g = params.plot_parameters(rows, cols, default_values)

Here you see the results of using different settings for 'blur' and 'color'. The settings for the other parameters are the ones you've specified as default.

To save the plots of all combinations of parameters, use the function below.

In [ ]:
params.batch_plot_parameters(default_values)

The figures are saved in the same directory as where the associated ParameterSelection dataframe is saved. You can go to the folder where the figures are saved and walk through the figures. That way you get a sense of the influence of various parameter-value combinations on location detection.

For certain parameters, you might see large noise differences for the different values. For these parameters, choose the best value (the one with the least noise), and use these values as default. Run the above cells again with the new default values. The figures will be saved in a new folder (figures_2). Look again at the figures. Do this several rounds, untill you get an idea of which parameter-value combinations provide the best (least noisy) location detection.

#### Superimpose on video

In the plots you get an idea of which paramater-value combinations result in the least noisy graphs. However, it is not possible to see whether the pixel coordinates also accurately match the location of the bird. For this, it is usefull to plot the mean coordinates directly on top of the video.

In [ ]:
params.get_parameters('multi_only')

Again, look at the parameters with multiple values. Choose from these parameters which values you would like to see plotted as circle on the videofragment.

In [ ]:
# choose which settings to superimpose on the videofragment
settings = {'resizebyfactor': [1.0],
            'color': [False],
            'VarThreshold': [30, 70],
            'blur': [0, 10],
            'morphologyex': [True],
            'History': [4],
            'VarMin': [10]}

all_combinations = list(md.product_dict(**settings))
print(f'There are {len(all_combinations)} combinations of settings to superimpose on a video.')

Too many circles plotted on the video are hard to follow. As default, a maximum of 6 circles can be superimposed on one videofragment, but often you'll probably want to plot less circles.

In [ ]:
# draw circles on videofragment
frames, colorspecs = params.draw_multiple_circles(settings)

In [ ]:
# show the settings for each color of the circles
colorspecs

In [ ]:
# look at the video using show()
frames.show(framerate=20)

TIP: a lower framerate makes it easier to follow the circles.

In [ ]:
# or, save as video with circles superimposed
vfs_circles = frames.tovideo(f'{params.path}/multicircles.mp4', framerate=params.vfs.avgframerate)

In [ ]:
# you can also save the color specification
colorspecs.to_csv(f'{params.path}/multicircles_colorspecs.csv')

Now, you have an idea which parameters have a large influence on movement detection. You might want to run the notebook again and test some other values or the parameters to fine-tune your results even more. Just repeat all the steps above.

Also, repeat these steps with a second short representative videofragment to make sure the same parameter-value combinations provide the best results. After that, you could use these settings to run movement detection on all your videos. For this, have a look at the next notebook!